# Pré traitement des données

## I. Conversion C3D en JSON rééchantilloné à 50 Hz

In [ ]:
import numpy as np
import ezc3d
import json
from scipy.signal import resample,butter, lfilter, freqz
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def recuperer_evenements(file_path):
    """
    Cette fonction extrait les événements du fichier C3D.

    Args:
        file_path (str): Le chemin du fichier C3D.

    Returns:
        dict: Un dictionnaire contenant les événements et leur temps correspondant.
    """
    c3d = ezc3d.c3d(file_path)
    events = c3d['parameters']['EVENT']['LABELS']['value'] # Récupérer les événements
    temps_events = c3d['parameters']['EVENT']['TIMES']['value'] # Récupérer les temps correspondant aux événements
    # Créer un DataFrame
    df = pd.DataFrame({'events': events, 'frames': temps_events[1]})
    # Supprimer l'index
    df.reset_index(drop=True, inplace=True)
    events_dict = df.groupby('events')['frames'].apply(list).to_dict() #self.events final
    return events_dict

def butter_lowpass(cutoff, fs, order=2):
    """
    Crée les coefficients d'un filtre Butterworth passe-bas.

    Args:
        cutoff (float): Fréquence de coupure du filtre.
        fs (float): Fréquence d'échantillonnage du signal.
        order (int): Ordre du filtre.

    Returns:
        tuple: Coefficients du filtre (b, a).
    """
    nyq = 0.5 * fs # Fréquence de Nyquist
    normal_cutoff = cutoff / nyq # Fréquence de coupure normalisée
    b, a = butter(order, normal_cutoff, btype='low', analog=False) # Création des coefficients du filtre
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    """
    Applique un filtre Butterworth passe-bas à un signal.

    Args:
        data (array_like): Signal à filtrer.
        cutoff (float): Fréquence de coupure du filtre.
        fs (float): Fréquence d'échantillonnage du signal.
        order (int): Ordre du filtre.

    Returns:
        array_like: Signal filtré.
    """
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def reechantillonnage_fc_coupure_et_association_labels_et_data(file_path, cutoff_freq=20, target_freq=50):
    """
    Cette fonction transforme les données échantillonnées à 2100 Hz en données échantillonnées à 50 Hz,
    puis applique un filtre Butterworth passe-bas avec une fréquence de coupure de 20 Hz, et enfin associe les étiquettes des données aux données rééchantillonnées et filtrées.

    Args:
        file_path (str): Le chemin du fichier C3D.
        cutoff_freq (float): Fréquence de coupure du filtre Butterworth (20 Hz).
        target_freq (float): Fréquence cible après rééchantillonnage (50 Hz).

    Returns:
        tuple: Un tuple contenant un dictionnaire fusionnant les étiquettes et les données rééchantillonnées et filtrées, et les temps rééchantillonnés.
    """
    c3d = ezc3d.c3d(file_path)
    labels = c3d['parameters']['ANALOG']['LABELS']['value'] # Récupérer les étiquettes des données
    original_freq = c3d['parameters']['ANALOG']['RATE']['value'][0] # Récupérer la fréquence d'échantillonnage
    data = c3d['data']['analogs'] # Récupérer les données
    nb_frame = len(data[0][0]) # Récupérer le nombre de frames
    target_freq = 50  # fréquence de rééchantillonnage
    
    # Rééchantillonnage des données
    nb_samples_target = int(nb_frame * (target_freq / original_freq)) # Calcul du nombre d'échantillons cible
    resampled_times = np.linspace(0., (nb_frame / original_freq), num=nb_samples_target) # Création d'un tableau de temps rééchantillonné
    resampled_data = np.zeros((len(labels), nb_samples_target)) # Création d'un tableau de zéros de la taille des données rééchantillonnées
    for i in range(len(labels)): # Pour chaque étiquette
        resampled_data[i, :] = resample(data[0][i, :], nb_samples_target) # Rééchantillonnage des données

    fusion_label_data = {}
    for i, label in enumerate(labels): # Pour chaque étiquette
        fusion_label_data[label] = resampled_data[i, :] # Associer les étiquettes aux données rééchantillonnées
        
    filtered_fusion_label_data = {}  # Initialiser un dictionnaire pour stocker les données filtrées

    for label, data in fusion_label_data.items():  # Itérer sur chaque étiquette et signal
        filtered_signal = butter_lowpass_filter(data, cutoff_freq, target_freq)  # Appliquer le filtre Butterworth
        filtered_fusion_label_data[label] = filtered_signal  # Stocker le signal filtré dans le dictionnaire de données filtrées
    
    return filtered_fusion_label_data, resampled_times
    

def filtrer_labels(filtered_fusion_label_data):
    """
    Cette fonction filtre les étiquettes des données pour ne garder que celles liées à l'accéléromètre et au gyroscope.

    Args:
        fusion_label_data (dict): Dictionnaire contenant les étiquettes et les données associées.

    Returns:
        tuple: Un tuple contenant une liste des étiquettes filtrées et un dictionnaire des données associées aux étiquettes filtrées.
    """
    labels_filtre = []
    labels_data_filtre = {}
    for label, valeurs in filtered_fusion_label_data.items(): # Pour chaque étiquette et ses valeurs
        if 'ACC' in label or 'GYRO' in label: # Si l'étiquette contient 'ACC' ou 'GYRO'
            labels_filtre.append(label) # Ajouter l'étiquette à la liste des étiquettes filtrées
            labels_data_filtre[label] = valeurs # Ajouter les valeurs associées à l'étiquette dans le dictionnaire des données filtrées
    return labels_filtre, labels_data_filtre

def calcul_norme(labels_data_filtre):
    """
    Cette fonction calcule les normes des données filtrées.

    Args:
        labels_data_filtre (dict): Dictionnaire contenant les étiquettes et les données associées, filtrées.

    Returns:
        dict: Un dictionnaire contenant les normes calculées.
    """
    normes = {}
    traite = set() # Créer un ensemble vide pour stocker les capteurs, les côtés et les mesures déjà traités
    for key, value in labels_data_filtre.items():
        parts = key.split('_') # Séparer l'étiquette en parties
        sensor = parts[0] # Récupérer le capteur
        side = parts[1] # Récupérer le côté
        measure = parts[2] # Récupérer la mesure (GYRO ou ACC)
        
        if (sensor, side, measure) not in traite: # Si le capteur, le côté et la mesure n'ont pas déjà été traités
            traite.add((sensor, side, measure)) # Ajouter le capteur, le côté et la mesure à l'ensemble des éléments traités
            
            if "ACC" in measure:
                # Obtention des axes X, Y et Z
                axe_X = (labels_data_filtre[f'{sensor}_{side}_{measure}_X'])
                axe_Y = (labels_data_filtre[f'{sensor}_{side}_{measure}_Y'])
                axe_Z = (labels_data_filtre[f'{sensor}_{side}_{measure}_Z'])
            
                norme = np.sqrt(axe_X**2 + axe_Y**2 + axe_Z**2) - 1 # Calcul de la norme auquelle on soustrait 1 pour enlever la gravité
                nom_cle = f'{sensor}_{side}_{measure}_norme'
                normes[nom_cle] = norme
                
            else:
                axe_X = labels_data_filtre[f'{sensor}_{side}_{measure}_X']
                axe_Y = labels_data_filtre[f'{sensor}_{side}_{measure}_Y']
                axe_Z = labels_data_filtre[f'{sensor}_{side}_{measure}_Z']
        
                norme = np.sqrt(axe_X**2 + axe_Y**2 + axe_Z**2) 
                nom_cle = f'{sensor}_{side}_{measure}_norme'
                normes[nom_cle] = norme

    return normes

def creer_structure_json(labels_data_filtre, patient_id, date_de_naissance, medicaments, resampled_times, events_dict, normes):
    """
    Cette fonction crée une structure JSON à partir des données filtrées et d'autres informations.

    Args:
        labels_data_filtre (dict): Dictionnaire contenant les étiquettes et les données associées, filtrées.
        patient_id (int): Identifiant du patient.
        date_de_naissance (str): Date de naissance du patient.
        medicaments (str): Médicaments pris par le patient.
        resampled_times (ndarray): Temps rééchantillonné.
        events_dict (dict): Dictionnaire contenant les événements et leur temps correspondant.
        normes (dict): Dictionnaire contenant les normes calculées.

    Returns:
        dict: Structure JSON contenant les données et les métadonnées.
    """
    json_data = {
        "metadata": {
            "Details du patient": {
                "Identifiant": patient_id,
                "Date de naissance": date_de_naissance,
                "Medicaments": medicaments
            },
            "Temps": resampled_times.tolist()
        }
    }
    
    for key, value in labels_data_filtre.items():
        parts = key.split('_')
        sensor = parts[1]
        side = parts[0]
        measure = parts[2]
        axis = parts[3]
        
        if sensor not in json_data:
            json_data[sensor] = {}

        if side not in json_data[sensor]:
            json_data[sensor][side] = {}

        if measure not in json_data[sensor][side]:
            json_data[sensor][side][measure] = {}

        json_data[sensor][side][measure][axis] = value.tolist()
        
    for key in normes:
        parts = key.split('_')
        sensor = parts[1]
        side = parts[0]
        measure = parts[2]
        axis = parts[3]

        if sensor not in json_data:
            json_data[sensor] = {}

        if side not in json_data[sensor]:
            json_data[sensor][side] = {}

        if measure not in json_data[sensor][side]:
            json_data[sensor][side][measure] = {}

        # Insérer la norme au même niveau d'indentation que l'axe
        json_data[sensor][side][measure][axis] = value.tolist()
        json_data[sensor][side][measure]["norme"] = normes[key].tolist()
    
        # # Ajouter les événements FOG
        # json_data["FOG"] = {
        #     "Debut": events_dict["FOG_begin"],
        #     "Fin": events_dict["FOG_end"]
        # }
    
        # # Ajouter tous les évènements sauf FOG
        # del events_dict["FOG_begin"]
        # del events_dict["FOG_end"]
            
    if "FOG_begin" in events_dict and "FOG_end" in events_dict:
        json_data["FOG"] = {
            "Debut": events_dict["FOG_begin"], 
            "Fin": events_dict["FOG_end"]
        }
        del events_dict["FOG_begin"]
        del events_dict["FOG_end"]
    else:
        json_data["FOG"] = {
            "Debut": [0],
            "Fin": [0]
        }
    
    json_data["Parcours"] = events_dict
    
    return json_data

def creation_json_grace_c3d(file_path, patient_id, date_de_naissance, medicaments, output_path):
    """
    Cette fonction crée un fichier JSON à partir d'un fichier C3D.

    Args:
        file_path (str): Le chemin du fichier C3D.
        patient_id (int): Identifiant du patient.
        date_de_naissance (str): Date de naissance du patient.
        medicaments (str): Médicaments pris par le patient.
        output_path (str): Le chemin de sortie du fichier JSON.
    """
    events_dict = recuperer_evenements(file_path) 
    filtered_fusion_label_data, resampled_times = reechantillonnage_fc_coupure_et_association_labels_et_data(file_path)
    labels_filtre, labels_data_filtre = filtrer_labels(filtered_fusion_label_data)
    normes = calcul_norme(labels_data_filtre)
    json_structure = creer_structure_json(labels_data_filtre, patient_id, date_de_naissance, medicaments, resampled_times, events_dict, normes)
    
    with open(output_path, "w") as fichier_json:
        json.dump(json_structure, fichier_json, indent=4)

# Utilisation de la fonction pour traiter le fichier C3D et générer le fichier JSON
fichier_brut = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/LE_LIEVRE_Emmanuel_1971_03_19_LEEM1971/2023-05-26/2023-05-26_overlay_detectFOG/Video overlay 16 - Anthony.c3d"
fichier_brut_6 = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/MORETTY_Daniel_1963-06-23_MODA1963/2023-07-07/2023-07-07_overlay_unspecified/Video overlay 6.c3d"
fichier_brut_16 = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/LE_LIEVRE_Emmanuel_1971_03_19_LEEM1971/2023-05-26/2023-05-26_overlay_detectFOG/Video overlay 6_start_stop.c3d"
fichier_brut_3 = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/MORETTY_Daniel_1963-06-23_MODA1963/2023-07-07/2023-07-07_overlay_unspecified/Video overlay 3.c3d"

patient_id = 1234
date_de_naissance = 45
medicaments = "ON"
chemin_fichier_json_6_start_stop = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 6_start_stop.json"
chemin__fichier_json = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 6.json"
chemin__fichier_json_3 = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 3.json"
creation_json_grace_c3d(fichier_brut_16, patient_id, date_de_naissance, medicaments, chemin_fichier_json_6_start_stop)


Très bizarre, malgré le fait qu'il n'y est aucun évènement de noté, le code me sort quand même des évènements. Cela est peut-être lié à des
étiquettes par défaut

In [ ]:
chemin_fichier_json_16 = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 16.json"
chemin_fichier_json_anthony = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 16 - Anthony.json"
chemin_fichier_json_6_start_stop = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 6_start_stop.json"
chemin__fichier_json = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 6.json"
chemin__fichier_json_3 = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/Video overlay 3.json"

# Charger le fichier JSON
with open(chemin_fichier_json_16, "r") as fichier_json:
    donnees_patient_16 = json.load(fichier_json)
    
with open(chemin_fichier_json_anthony, "r") as fichier_json:
    donnees_patient_anthony = json.load(fichier_json) 
    
with open(chemin__fichier_json, "r") as fichier_json:
    donnees_patient_1 = json.load(fichier_json)   

with open(chemin_fichier_json_6_start_stop, "r") as fichier_json:
    donnees_patient_16_update = json.load(fichier_json)   
    
with open(chemin__fichier_json_3, "r") as fichier_json:
    donnees_patient_3 = json.load(fichier_json) 

# Accéder aux données spécifiques
#temps = donnees_patient["Rectus Femoris"]["Left"]["ACC"]["norme"]
#temps_16 = donnees_patient_16["metadata"]["Temps"]
#Norme_GYRO_tibia_16 = donnees_patient_16["Tibialis Anterior"]["Left"]["GYRO"]["norme"]
#FOG_overlay_16 = donnees_patient_16["FOG"]
#parcours_16 = donnees_patient_16["Parcours"]

## Normaliser les données

In [ ]:
def extract_data_interval(data):
    """
    Extrait un intervalle de données à partir du moment où le patient se lève jusqu'au dernier moment debout.

    Parameters:
    - data (dict): Un dictionnaire contenant les données des capteurs.

    Returns:
    - data_interval (dict): Un dictionnaire contenant l'intervalle de données extrait.
    """
    # Extraire les temps de début et de fin du parcours
    start_time = data["Parcours"]["START"][0] # Extraire le temps de début du parcours
    end_time = data["Parcours"]["END"][0] # Extraire le temps de fin du parcours
    epsilon=0.01 # Marge d'erreur

    # Trouver les indices correspondants dans le vecteur de temps pour le début du parcours
    for i, time in enumerate(data["metadata"]["Temps"]): # Pour chaque temps dans le vecteur de temps
        if abs(time - start_time) < epsilon:  # Vérifier si la différence est inférieure à la marge d'erreur
            start_index = i 
        if abs(time - end_time) < epsilon:
            end_index = i

    # Extraire les données d'axes pour la plage de temps START à END
    data_interval = {}
    for sensor, sensor_data in data.items():
        if sensor not in ["metadata", "Parcours", "FOG"]:
            data_interval[sensor] = {}
            for side, side_data in sensor_data.items():
                data_interval[sensor][side] = {}
                for measure, measure_data in side_data.items():
                    data_interval[sensor][side][measure] = {}
                    for axis, axis_data in measure_data.items():
                        data_interval[sensor][side][measure][axis] = axis_data[start_index:end_index+1]

    # Copier les données de "metadata", "Parcours" et "FOG"
    data_interval["metadata"] = data["metadata"]
    data_interval["Parcours"] = data["Parcours"]
    data_interval["FOG"] = data["FOG"]

    # Extraire la plage de temps START à END pour la liste de temps dans metadata
    metadata_temps_interval = data["metadata"]["Temps"][start_index:end_index+1]

    # Ajouter la plage de temps interval à metadata
    data_interval["metadata"]["Temps"] = metadata_temps_interval

    return data_interval

# Utilisation de la fonction
data_interval = extract_data_interval(donnees_patient_16_update)
data_interval_16 = extract_data_interval(donnees_patient_16)



## On applique la formule de normalisation

In [ ]:
def normalize_data(data):
    """
    Normalise les données des capteurs.

    Parameters:
    - data (dict): Un dictionnaire contenant les données des capteurs.

    Returns:
    - normalized_data (dict): Un dictionnaire contenant les données normalisées.
    """
    normalized_data = {}
    for sensor, sensor_data in data.items():
        if sensor not in ["metadata", "Parcours", "FOG"]:
            normalized_data[sensor] = {}
            for side, side_data in sensor_data.items():
                normalized_data[sensor][side] = {}
                for measure, measure_data in side_data.items():
                    normalized_data[sensor][side][measure] = {}
                    for axis, axis_data in measure_data.items():
                        # Calculer la moyenne, le maximum et le minimum des données
                        mean = np.mean(axis_data)
                        max_val = np.max(axis_data)
                        min_val = np.min(axis_data)
                        # Appliquer la normalisation sur tous les axes X,Y,Z et la norme
                        normalized_axis_data = (axis_data - mean) / (max_val - min_val)
                        normalized_data[sensor][side][measure][axis] = normalized_axis_data
    
    # Copier les données de "metadata", "Parcours" et "FOG"
    normalized_data["metadata"] = data["metadata"]
    normalized_data["Parcours"] = data["Parcours"]
    normalized_data["FOG"] = data["FOG"]
    
    return normalized_data


normalized_data = normalize_data(data_interval)
normalized_data_16 = normalize_data(data_interval_16)


# fenêtrage

In [ ]:
def decoupage_en_fenetres(data, taille_fenetre, decalage, taux_echantillonnage):
    fenetres_data = {}
    infos_fenetres = {}

    for sensor, sensor_data in data.items():
        if sensor not in ["metadata", "Parcours", "FOG"]:
            fenetres_data[sensor] = {}
            infos_fenetres[sensor] = {}

            for side, side_data in sensor_data.items():
                fenetres_data[sensor][side] = {}
                infos_fenetres[sensor][side] = {}

                for measure, measure_data in side_data.items():
                    fenetres_data[sensor][side][measure] = {}
                    infos_fenetres[sensor][side][measure] = {}

                    for axis, axis_data in measure_data.items():
                        taille_signal = len(axis_data)
                        taille_fenetre_echantillons = int(taille_fenetre * taux_echantillonnage)
                        decalage_fenetre = int(decalage * taille_fenetre_echantillons)

                        fenetres = []
                        debut = 0
                        fin = taille_fenetre_echantillons
                        nb_fenetres = 0

                        while fin <= taille_signal:
                            fenetre = axis_data[debut:fin]
                            fenetres.append(fenetre)

                            debut = debut + decalage_fenetre
                            fin = fin + decalage_fenetre
                            nb_fenetres += 1

                        if debut < taille_signal:
                            fenetre = axis_data[debut:]
                            fenetres.append(fenetre)

                        fenetres_data[sensor][side][measure][axis] = fenetres
                        infos_fenetres[sensor][side][measure][axis] = {
                            "nombre_fenetres": nb_fenetres,
                            "taille_fenetre": taille_fenetre_echantillons,
                            "decalage_fenetre": decalage_fenetre
                        }

    # Traitement des nouvelles données de temps
    temps = data["metadata"]["Temps"]
    taille_signal_temps = len(temps)
    taille_fenetre_temps = int(taille_fenetre * taux_echantillonnage)
    decalage_fenetre_temps = int(decalage * taille_fenetre_temps)

    fenetres_temps = []
    debut_temps = 0
    fin_temps = taille_fenetre_temps

    while fin_temps <= taille_signal_temps:
        fenetre_temps = temps[debut_temps:fin_temps]
        fenetres_temps.append(fenetre_temps)

        debut_temps += decalage_fenetre_temps
        fin_temps += decalage_fenetre_temps

    if debut_temps < taille_signal_temps:
        fenetre_temps = temps[debut_temps:]
        fenetres_temps.append(fenetre_temps)

    # Copie des données de "metadata", "Parcours" et "FOG"
    fenetres_data["metadata"] = data["metadata"]
    fenetres_data["Parcours"] = data["Parcours"]
    fenetres_data["FOG"] = data["FOG"]
    
    # Remplacement des anciennes données de temps par les nouvelles
    fenetres_data["metadata"]["Temps"] = fenetres_temps

    return fenetres_data, infos_fenetres

fenetres_data, infos_fenetres = decoupage_en_fenetres(normalized_data, taille_fenetre = 2, decalage=0.2, taux_echantillonnage=50)
fenetres_data_16, infos_fenetres_16 = decoupage_en_fenetres(normalized_data_16, taille_fenetre = 2, decalage=0.2, taux_echantillonnage=50)

# Labelisation

In [ ]:
def label_fenetre(data,temps_prefog = 3):
    temps = data["metadata"]["Temps"]
    debuts_fog = data["FOG"]["Debut"]
    fins_fog = data["FOG"]["Fin"]
    debuts_prefog = [x-temps_prefog for x in debuts_fog]
    status="NoFOG"
    
    # on stock les données d'évènement dans un dataframe ordonner en fonction du temps
    events=pd.DataFrame({'temps': debuts_fog + fins_fog + debuts_prefog, 
                    'events': ["debut_fog"]*len(debuts_fog) + ["fin_fog"]*len(fins_fog) + ["preFog"]*len(debuts_prefog)}).sort_values('temps').reset_index(drop=True)

    # On récupère si il y a un évènement de FOG ou plusieurs de présent de la FOG
    statuses = []
    status = "NoFog"

    for window in temps:
        w_start = window[0] # premier terme de la fenêtre
        w_end = window[-1] # dernier terme de la fenêtre
        window_events = []  # Liste pour stocker les événements de la fenêtre
        time=[]
        time_pourcent = 1 
        
        for _, row in events.iterrows():
            if row['temps'] >= w_start and row['temps'] <= w_end: #si le temps correspondant à l'évènement se trouve entre début et fin de la fenêtre
                window_events.append(row['events'])  # Ajouter l'événement à la liste
                if row['events']== "fin_fog": # si l'évènement est fin_fog
                    time.append(row["temps"])
                                    
        if len(window_events)==1 and "fin_fog" in window_events: #si on oa une liste avec uniquement fin_fog
             time_array = np.arange(w_start,w_end,1/50)
             time_pourcent = np.sum(time_array<=time)/100
    
        if not window_events:  # Si la liste est vide
            window_events = [None]  # Remplir avec None
    
        # if status == "NoFog" and "preFog" in window_events:
        #     status = "transitionPreFog"
        
        # elif status == "transitionPreFog" and None in window_events:
        #     status = "preFog"
            
        if status == "NoFog" and "debut_fog" in window_events: # si la fenêtre contient debut_fog et son statu est NoFog
            status = "transitionFog"
            
        elif status == "transitionFog" and None in window_events: #si le FOG est suffisement long pour ne pas rencontrer d'évènement après debut_Fog alors :
            status = "Fog"
        
        elif "debut_fog" in window_events and "fin_fog" in window_events: #si il est petit alors la fenêtre peut comporter l'évènement de début et de fin
            status = "Fog"
            
        elif status =="Fog" and ("debut_fog" in window_events and "fin_fog" in window_events): # dans le cas où des FOG sont succints, donc c'est à dire quand la fenêtre comporte fin_fog et debut du fog suivant alors :  
            status= "transitionFog"
        
        elif status == "Fog" and "fin_fog" in window_events and time_pourcent <= 0.5: # si on a un FOG inférieur à 50% de la longueur de fenêtre, alors : 
        # on s'en fiche de faire cette opération avant, car dans tous les cas on considère FOG lorsqu'il y a deux évènements dans la fenêtre et on prend pour cible transitionFog,donc que ce soit FOG ou transition ce sera dans cible. 
            status = "transitionNoFog"
            
        elif status == "transitionNoFog" and None in window_events:
            status = "NoFog"
        
        statuses.append(status)  # Ajouter le statut à la liste des statuts
        
    # on associe les labels de fenêtre dans notre data
    data["labels_fenetres"] = statuses
    return data

In [ ]:
statuses_16=label_fenetre(fenetres_data_16)
print(statuses_16["labels_fenetres"])
satuses=label_fenetre(fenetres_data)
c= statuses_16["labels_fenetres"]
print(c)

#### Maintenant j'aimerais définir le label de chaque fenêtre en clé de données

In [ ]:
def association_label_fenetre_data(satuses):
    association_label_fenetre_data = {}
    for sensor, sensor_data in satuses.items():
        if sensor not in ["metadata", "Parcours", "FOG", "labels_fenetres"]:
            association_label_fenetre_data[sensor] = {}
            for side, side_data in sensor_data.items():
                association_label_fenetre_data[sensor][side] = {}

                for measure, measure_data in side_data.items():
                    association_label_fenetre_data[sensor][side][measure] = {}
                    
                    for axis, axis_data in measure_data.items():
                        association_label_fenetre_data[sensor][side][measure][axis] = {}
                        presentWin=np.unique(satuses["labels_fenetres"])
                        for label in presentWin:
                            #data_list = axis_data
                            data_frame_axis_data= pd.DataFrame(axis_data)
                            association_label_fenetre_data[sensor][side][measure][axis][label]=data_frame_axis_data[[x==label for x in satuses["labels_fenetres"]]]
    
        # Copie des données de "metadata", "Parcours" et "FOG"
    association_label_fenetre_data["metadata"] = satuses["metadata"]
    association_label_fenetre_data["Parcours"] = satuses["Parcours"]
    association_label_fenetre_data["FOG"] = satuses["FOG"]

    association_label_fenetre_data["FOG"] = {
        "Debut": satuses["FOG"]["Debut"],
        "Fin": satuses["FOG"]["Fin"],
        "preFog": [max(0, x - 3) for x in satuses["FOG"]["Debut"]]
    }
    return association_label_fenetre_data 

In [ ]:
temp=association_label_fenetre_data(satuses)
temp_16=association_label_fenetre_data(statuses_16)
print(temp["Tibialis Anterior"]["Left"]["GYRO"]["norme"]["NoFog"])
print(temp["FOG"]["Debut"])

print(temp)

In [ ]:
# Statistiques

In [ ]:
temp_16["metadata"]["Temps"] = pd.DataFrame(temp_16["metadata"]["Temps"])

In [ ]:
a = temp_16["metadata"]["Temps"]

In [ ]:
first_time= temp_16["metadata"]["Temps"].iloc[0,0]
if np.isnan(temp_16["metadata"]["Temps"].iloc[-1,-1]):
    first_na = np.where(np.isnan(temp_16["metadata"]["Temps"]))[-1][0]
    last_time = temp_16["metadata"]["Temps"].iloc[-1,(first_na-1)]
else:
    last_time = temp_16["metadata"]["Temps"].iloc[-1,-1]

print(last_time)


temps_total = last_time - first_time
print("Temps total de l'enregistrement :", temps_total, "secondes")

if  temp_16["FOG"]["Debut"] == [0] :
    nb_fog = 0
    print(f"Nombre de FOG : {nb_fog}")
else :
    nb_fog = len(temp_16["FOG"]["Debut"])
    print(f"Nombre de FOG : {nb_fog}")

temps_fog = sum([fin - debut for debut, fin in zip(temp_16["FOG"]["Debut"], temp_16["FOG"]["Fin"])])
prct_fog = (temps_fog / temps_total) * 100
print(f"Pourcentage total de FOG sur la totalité de l'enregistrement : {prct_fog:.2f}%")



In [ ]:
temps_test =temp_16["metadata"]["Temps"]
nb_fenetre = len(temp_16["metadata"]["Temps"])
nb_colonne = len(temp_16["metadata"]["Temps"].columns)
print(f"Nombre de fenêtres : {nb_fenetre}")
print(f"Nombre de colonnes : {nb_colonne}")

In [ ]:
tab_stat= pd.DataFrame({"Temps total de l'enregistrement": [temps_total], "Nombre de FOG": [nb_fog], "Pourcentage total de FOG": [prct_fog], "nombre de fenêtres": [nb_fenetre], "longueur des fenêtres": [nb_colonne]})
print(tab_stat)


In [ ]:

temps_fog = [fin - debut for debut, fin in zip(temp_16["FOG"]["Debut"], temp_16["FOG"]["Fin"])]
tab_fog = pd.DataFrame({"Debut": temp_16["FOG"]["Debut"], "Fin": temp_16["FOG"]["Fin"], "Total" : temps_fog})
print(tab_fog)

In [ ]:
temp_16["metadata"]["Temps"] = pd.DataFrame(temp_16["metadata"]["Temps"])
print(temp_16["metadata"]["Temps"].iloc[0,0])
# Trouver l'indice de la dernière valeur non nulle
# Trouver la dernière valeur non nulle avant NaN dans la dernière colonne
first_na=np.where(np.isnan(temp_16["metadata"]["Temps"]))[-1][0]
last_time=temp_16["metadata"]["Temps"].iloc[-1,(first_na-1)]
print(last_time)